In [ ]:
!pip install transformers
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import numpy as np
import torch
import transformers as ppb 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text


In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def gpt_fscore(text,model,tokenizer):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0) 
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    return last_hidden_states

In [ ]:
fn=['train']
for k in range(0,1):
    fname='/content/'+fn[k]+'.csv'
    df = pd.read_csv(fname,encoding='latin-1')
    df['tweet'] = df['tweet'].apply(clean_text)
    print(df.head(10))
    train_tokens = list(map(lambda t:   tokenizer.tokenize(t)+['[CLS]'], df['tweet']))
    train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,train_tokens))
    b=np.zeros((np.shape(df)[0]))
    for i in range(0,np.shape(df)[0]):
        b[i]=np.shape(train_tokens_ids[i])[0]
    pad=np.zeros((np.shape(df)[0],int(np.max(b))))
    for i in range(0,np.shape(df)[0]):
        a=train_tokens_ids[i]
        for j in range(0,np.shape(a)[0]):
            pad[i,j]=a[j]
    
    pad1=pad[:,:]       
    input_ids = torch.tensor(np.array(pad1))        
    with torch.no_grad():
        last_hidden_states = model(input_ids.long())    
    features = last_hidden_states[0][:,0,:].numpy()
    fname='/content/gpt2_'+fn[k]+'.csv'
    np.savetxt(fname,features, delimiter=',', fmt='%f') 

   id  label                                              tweet
0   1      0  user father dysfunctional selfish drags kids d...
1   2      0  user user thanks #lyft credit cant use cause d...
2   3      0                                     bihday majesty
3   4      0                       #model love u take u time ur
4   5      0                     factsguide society #motivation
5   6      0  2 2 huge fan fare big talking leave chaos pay ...
6   7      0  user camping tomorrow user user user user user...
7   8      0  next school year year exams cant think #school...
8   9      0  love land #allin #cavs #champions #cleveland #...
9  10      0                          user user welcome im #gr8
